In [1]:
%load_ext autoreload
%autoreload 2

import matplotlib
#matplotlib.use('Agg')
%matplotlib tk
%autosave 180

#
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

#
import matplotlib.cm as cm
from matplotlib import gridspec
from scipy import signal

#
import numpy as np
import pandas as pd
import os
import shutil
import cv2
from tqdm import trange

# 
from scipy.io import loadmat
import scipy
import h5py

# 
import csv
import sleap

# 
from visualize import Visualize
from track import Track


Autosaving every 180 seconds


In [3]:
###################################################
###################################################
###################################################
# 
def save_two_animal_slp(animal_ids, fname_slp):
    # select first to animals
    labels = sleap.load_file(fname_slp)

    # Change this to specify which pair of tracks to save:
    keep_tracks = [labels.tracks[animal_ids[0]], labels.tracks[animal_ids[1]]]

    # Remove instances that aren't in keep_tracks
    for lf in labels:
        lf.instances = [inst for inst in lf.instances if inst.track in keep_tracks]

    # Keep only those tracks in the labels
    labels.tracks = keep_tracks

    # Remove frames that are now empty due to the track filtering
    labels.remove_empty_frames()

    # Save the result with the specified tracks in the filename
    labels.save(fname_slp[:-4] + "_"+str(animal_ids[0])+"_"+str(animal_ids[1])+".slp")
    
#
fname_slp = '/media/cat/256GB/dan/simba/2020_07_31_20_15_17_668937_compressed_defished_shrink_cropped.slp'
fname_vid = '/media/cat/256GB/dan/simba/2020_07_31_20_15_17_668937_compressed_defished_shrink_cropped.mp4'

#
animal_ids = [0,1]
save_two_animal_slp(animal_ids, fname_slp)

#


In [5]:
######################################################## 
############## GENERATE FIXED TRACK MOVIE ##############
########################################################
#
fps = 25
shrink = 1 # shrink image factor
fname_video_out = 'fixed'
fname_video = '/media/cat/256GB/dan/simba/2020_07_31_20_15_17_668937_compressed_defished_shrink_cropped.mp4'

# 
track = Track('/media/cat/256GB/dan/simba/2020_07_31_20_15_17_668937_compressed_defished_shrink_cropped_0_1.slp')
track.animal_ids = [0,1]
track.tracks_names = ['female','male']
track.recompute_spine_centres=True
track.verbose = True                      # gives additional printouts
track.n_animals = 2                       # number of animals
track.filter_width = 10                   # this is the median filter width in frames; e.g. 10 ~=0.4 seconds
                                          # higher values provide more stability, but less temporally precise locations
# 
track.load_tracks()

######################################################
### OPTIONAL 1 - MEDIAN FILTER ALL BODY PARTS ########
######################################################
if False:
    track.filter_tracks()
            
######################################################
### OPTIONAL 2 - ALGORITHM TO REASSIGN CHUNKING ######
######################################################
if False:
    # parameters for fixing track chunking

    track.max_jump_single_frame = 100   # max distance that an animal can move in a single frame
    track.max_time_automerge = 3        # time to automerget chunks from same animal ???
    track.max_dist_automerge = 25       # distance to auto merge chunks from same animal separated by single time skip

    # 
    track.get_scores()

    #
    track.make_tracks_chunks()

    # 
    track.time_threshold = 25          # window to search for nearest chunks
    track.safe_chunk_length = 15       # chunks this long will not change id
    track.min_chunk_len = 4            # min length of chukn to be used for anchoring/correcting
    track.max_distance_merge = 75      # max pix diff allowed for merging when using model
    
    # track.memory_length = 25         # how many frames back is it ok to remember a prev animal
    track.verbose = False
    track.update_tracks = True

    track.fix_tracks()

else:
    # recompute spine centres from scratch
    track.get_track_spine_centers()
    
######################################################
### OPTIONAL 3 - MEDIAN FILTER SPINE CENTRES #########
######################################################
if False:
    track.filter_tracks_centres() 
    
####################################################
########## MAKE MOVIES OF SPINE CENTRES ############
####################################################
vis = Visualize()
start = 0   # frame to start
end = 2000   # frame to end
vis.make_video_centroid(track.tracks_spine,                # visualize centres locations
                        fname_video,
                        fname_video_out,
                        start,
                        end,
                        fps,
                        shrink) 


# 

  0%|          | 9/2000 [00:00<00:24, 81.75it/s]

width, heigh:  900.0 700.0
Histories:  (2, 5, 2)


100%|██████████| 2000/2000 [00:23<00:00, 83.91it/s]
